In this notebook we use pretrain Gemma model with two different versions of prompt engineering.

# PREREQUISITES:

In [4]:
import pandas as pd
from transformers import AutoModelForCausalLM
from sklearn.model_selection import train_test_split
import torch
from sklearn.model_selection import train_test_split
from transformers import GemmaTokenizerFast, pipeline
from sklearn.metrics import accuracy_score
from huggingface_hub import login
import json
import numpy as np
import os

In [2]:
login()

In [3]:
current_directory = os.getcwd() # get current directory
file_path = os.path.join(current_directory, '..', '..', 'data', 'processed', 'clean_data.csv') # navigate to folder with preprocessed data
data = pd.read_csv(file_path)
print(data.head(5))

                                        heading_text  bias_rating
0  chicago gun violence spikes and increasingly f...            0
1  'bullets just came from nowhere': fourth of ju...            1
2  dozens of shootings across us mark bloody july...            2
3  federal government will run out of cash on oct...            2
4  yellen tells congress that u.s. will run out o...            0


# Prompts


Prompt 1 - is a standard where we include minimal details, we just specify the task in each section


Prompt 2 - is more detailed and elaborate, includes logical reasoning

In [6]:
prompt_1 = """This is a political bias classifier for news headlines.
Please provide the response in the following JSON format:
{
    "Clues": "<List the CLUES here (i.e., keywords, tones, references, contextual information)>",
    "Reasoning": "<Provide the diagnostic REASONING process here in 130 words based on the clues and input>",
    "Political Bias": "<Determine the overall POLITICAL BIAS as Left or Right. You MUST write only one word in this section either right or left.>"
}
INPUT: """

prompt_2 = """This is a political bias classification task for news headlines.
Your goal is to analyze the headline to identify relevant information and logically deduce the political bias.
Please provide the response in the following JSON format:
{
    "Clues": "<Identify specific CLUES (such as keywords, tones, references, or contextual elements) from the headline that can be used to determine the political bias.>",
    "Reasoning": "<Explain the REASONING behind the bias determination, detailing how the clues support your conclusion. Limit the reasoning to 130 words.>",
    "Political Bias": "<Determine the overall POLITICAL BIAS as Left, Right, or Center. You MUST write only one word in this section either right, left or center.>"
}
INPUT: """

In [9]:
# !!! Choose version of prompt you want to use
# 1 - version 1
# 2 - version 2
prompt_choice = 2

In [10]:

# Sentiment as a driving force
if prompt_choice == 2:
    df_prompt_2 = data.copy()
    df_prompt_2['heading_text'] = df_prompt_2['heading_text'].apply(lambda x: prompt_2 + x)
elif prompt_choice == 1:
    df_prompt_2 = data.copy()
    df_prompt_2['heading_text'] = df_prompt_2['heading_text'].apply(lambda x: prompt_1 + x)

print(df_prompt_2.iloc[0]['heading_text'])

This is a political bias classification task for news headlines.
Your goal is to analyze the headline to identify relevant information and logically deduce the political bias.
Please provide the response in the following JSON format:
{
    "Clues": "<Identify specific CLUES (such as keywords, tones, references, or contextual elements) from the headline that can be used to determine the political bias.>",
    "Reasoning": "<Explain the REASONING behind the bias determination, detailing how the clues support your conclusion. Limit the reasoning to 130 words.>",
    "Political Bias": "<Determine the overall POLITICAL BIAS as Left, Right, or Center. You MUST write only one word in this section either right, left or center.>"
}
INPUT: chicago gun violence spikes and increasingly finds the youngest victims as yasmin miller drove home from a laundromat in chicago's englewood neighborhood last weekend, a gunman in another car peppered her red hyundai sedan with bullets, grazing her head and st

# Data splitting

In [16]:
X = df_prompt_2['heading_text'].values
y = df_prompt_2['bias_rating'].values

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load Model and Tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")
tokenizer = GemmaTokenizerFast.from_pretrained('google/gemma-2b-it')

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Evaluate model

In [24]:
pipe = pipeline(
    "text-generation",
    tokenizer=tokenizer,
    model=model,
    device="cuda",
)
results = []
counter = 0

for text in X_test:
    counter += 1
    print(f"{counter}/{len(X_test)}")
    outputs = pipe(text, max_new_tokens=256)
    response = outputs[0]["generated_text"]
    
    if "**Clues:**" in response and "**Reasoning:**" in response and "**Political Bias:**" in response:
        clues = response.split("**Clues:**")[1].split("**Reasoning:**")[0].strip()
        reasoning = response.split("**Reasoning:**")[1].split("**Political Bias:**")[0].strip()
        bias = response.split("**Political Bias:**")[1].strip()
    else:
        clues = "NaN"
        reasoning = "NaN"
        bias = "NaN"

    result = {
        "Clues": clues,
        "Reasoning": reasoning,
        "Political Bias": bias
    }
    results.append(result)

if prompt_choice ==2:
    with open('political_bias_results_prompt_2.json', 'w') as f:
        json.dump(results, f, indent=4)
elif prompt_choice == 1:
    with open('political_bias_results_prompt_1.json', 'w') as f:
        json.dump(results, f, indent=4)

1/4349
2/4349
3/4349
4/4349
5/4349
6/4349
7/4349
8/4349
9/4349
10/4349
11/4349
12/4349
13/4349
14/4349
15/4349
16/4349
17/4349
18/4349
19/4349
20/4349
21/4349
22/4349
23/4349
24/4349
25/4349
26/4349
27/4349
28/4349
29/4349
30/4349
31/4349
32/4349
33/4349
34/4349
35/4349
36/4349
37/4349
38/4349
39/4349
40/4349
41/4349
42/4349
43/4349
44/4349
45/4349
46/4349
47/4349
48/4349
49/4349
50/4349
51/4349
52/4349
53/4349
54/4349
55/4349
56/4349
57/4349
58/4349
59/4349
60/4349
61/4349
62/4349
63/4349
64/4349
65/4349
66/4349
67/4349
68/4349
69/4349
70/4349
71/4349
72/4349
73/4349
74/4349
75/4349
76/4349
77/4349
78/4349
79/4349
80/4349
81/4349
82/4349
83/4349
84/4349
85/4349
86/4349
87/4349
88/4349
89/4349
90/4349
91/4349
92/4349
93/4349
94/4349
95/4349
96/4349
97/4349
98/4349
99/4349
100/4349
101/4349
102/4349
103/4349
104/4349
105/4349
106/4349
107/4349
108/4349
109/4349
110/4349
111/4349
112/4349
113/4349
114/4349
115/4349
116/4349
117/4349
118/4349
119/4349
120/4349
121/4349
122/4349
123/4349
1

# Calculate accuracy

In [11]:
if prompt_choice == 1:
    prompt_file = 'political_bias_results_prompt_1.json'
elif prompt_choice == 2:
    prompt_file = 'political_bias_results_prompt_2.json'

file_path = os.path.join(current_directory, '..', '..', 'results', prompt_file)

with open(file_path, 'r') as f:
    data = json.load(f)
    
bias_mapping = {'Left': 0, 'Center':1, 'Right': 2}

In [12]:
json_labels = []
for item in data:
    bias = item.get("Political Bias")
    
    if bias in bias_mapping:
        json_labels.append(bias_mapping[bias])
    elif isinstance(bias, str):
        bias_lower = bias.lower()
        if "left" in bias_lower:
            json_labels.append(bias_mapping['Left'])
        elif "right" in bias_lower:
            json_labels.append(bias_mapping['Right'])
        elif "center" in bias_lower:
            json_labels.append(bias_mapping['Center'])
        else:
            json_labels.append(-1)
    else:
        json_labels.append(-1)

json_array = np.array(json_labels)

In [21]:
mask = json_array != -1
json_array_filtered = json_array[mask] # remove NaN (-1) values from the dataset
y_test_filtered = y_test[mask] # remove values corresponding to NaN (-1) values from the set produced by model

print(len(json_array_filtered))
print(len(y_test_filtered))

accuracy = accuracy_score(y_test_filtered, json_array_filtered)
print(f'Accuracy: {accuracy * 100:.2f}%')

3895
3895
Accuracy: 37.48%
